### This program uses an artificial recurrent neural network called Long Short Term Memory (LSTM), to predict the closing stock price using the past 60 days stock price.

### Import Library

In [11]:
import math
import pandas_datareader as web
import numpy
import pandas
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as pyplot
import datetime

pyplot.style.use('fivethirtyeight')